In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicting-depression-machine-learning-challenge/sample_submission.csv
/kaggle/input/predicting-depression-machine-learning-challenge/train.csv
/kaggle/input/predicting-depression-machine-learning-challenge/test.csv


In [3]:
train = pd.read_csv('/kaggle/input/predicting-depression-machine-learning-challenge/train.csv')
test = pd.read_csv('/kaggle/input/predicting-depression-machine-learning-challenge/test.csv')

submission = pd.read_csv('/kaggle/input/predicting-depression-machine-learning-challenge/sample_submission.csv')

In [5]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

In [6]:
# Define target variable (change 'target_column' to the actual target name)
target = "Depression"

# Define feature columns (exclude target column)
features = [col for col in train.columns if col != target]


In [7]:
# Identify categorical columns
categorical_features = train.select_dtypes(include=['object']).columns.tolist()

# Convert categorical columns to string (CatBoost prefers them as strings)
train[categorical_features] = train[categorical_features].astype(str)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    train[features], train[target], test_size=0.2, random_state=42
)


In [10]:
train_pool = Pool(X_train, label=y_train, cat_features=categorical_features)
test_pool = Pool(X_test, label=y_test, cat_features=categorical_features)


In [11]:
model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, verbose=100)
model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50)


0:	learn: 0.5354043	test: 0.5354166	best: 0.5354166 (0)	total: 205ms	remaining: 1m 42s
100:	learn: 0.1474022	test: 0.1485016	best: 0.1485016 (100)	total: 10.4s	remaining: 41.3s
200:	learn: 0.1427325	test: 0.1472596	best: 0.1472596 (200)	total: 20.6s	remaining: 30.7s
300:	learn: 0.1394985	test: 0.1468827	best: 0.1468817 (299)	total: 30.6s	remaining: 20.2s
400:	learn: 0.1367588	test: 0.1467427	best: 0.1467410 (398)	total: 40.8s	remaining: 10.1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1466957759
bestIteration = 430

Shrink model to first 431 iterations.


In [12]:
y_pred = model.predict(X_test)


In [13]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.9396


In [14]:
# Identify categorical columns (if applicable)
categorical_features_test = test.select_dtypes(include=['object']).columns.tolist()

# Convert categorical columns to string (CatBoost prefers categorical variables as strings)
test[categorical_features_test] = test[categorical_features_test].astype(str)


In [17]:
# Make predictions
predictions = model.predict(test)


In [18]:
predictions

array([0, 0, 0, ..., 0, 1, 0])

In [20]:
submission

,id,Depression
0,140700,0
1,140701,0
2,140702,0
3,140703,0
4,140704,0
...,...,...
93795,234495,0
93796,234496,0
93797,234497,0
93798,234498,0


In [21]:
submission["Depression"] = predictions

In [22]:
submission

,id,Depression
0,140700,0
1,140701,0
2,140702,0
3,140703,1
4,140704,0
...,...,...
93795,234495,0
93796,234496,1
93797,234497,0
93798,234498,1


In [23]:
submission["Depression"].value_counts()

Depression
0    76975
1    16825
Name: count, dtype: int64

In [25]:
submission.to_csv("Submission_1.csv",index=False)